In [1]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 3.4MB/s 
     |████████████████████████████████| 552kB 42.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=ce0f811851d3690956ccc7d2e7214c05865d6225ebcc19649cd7cdd0f77535a2
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=2a2cb88ea026049d0c36d8d9eac66db7b9948355b456ca242fd03b8476510829
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [2]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("brown")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.corpus import brown as corpus

In [4]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .  
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise  
and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged  
by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan  
Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in  
the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia's  
registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislators act `

In [5]:
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:5])
print("num of docs:", len(docs))

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...], ['Austin', ',', 'Texas', '--', 'Committee', 'approval', ...], ['Several', 'defendants', 'in', 'the', 'Summerdale', ...], ['Oslo', 'The', 'most', 'positive', 'element', 'to', ...], ['East', 'Providence', 'should', 'organize', 'its', ...]]
num of docs: 500


In [7]:
en_stop = nltk.corpus.stopwords.words('english')
import re
stop_norm = [r".*?`.*?", r".*?'.*?", r'.*?".*?', r".*?-.*?", r".*?<.*?", r".*?>.*?", r".*?\(.*?", r".*?\).*?", r".*?:.*?", r".*?;.*?", r".*?\..*?", r".*?0.*?", r".*?1.*?", r".*?2.*?", r".*?3.*?", r".*?4.*?", r".*?5.*?", r".*?6.*?", r".*?7.*?", r".*?8.*?", r".*?9.*?"]

In [8]:
from nltk.corpus import wordnet as wn

def preprocess_word(word, stopwordset):
    
    word=word.lower()
    
    if word in [",","."]:
        return None
    
    if word in stopwordset:
        return None

    for r in stop_norm:
      if re.match(r,word):
        return None 
    
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset:
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [9]:
print(docs[0][:25]) 

print(preprocess_documents(docs)[0][:25])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
['fulton', 'county', 'grand', 'jury', 'say', 'friday', 'investigation', 'recent', 'primary', 'election', 'produce', 'evidence', 'irregularity', 'take', 'place', 'jury', 'say', 'presentment', 'city', 'executive', 'committee', 'charge', 'election', 'deserve', 'praise']


In [10]:
import gensim
from gensim import corpora

In [11]:
dictionary = corpora.Dictionary(preprocess_documents(docs))
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [12]:
print(dictionary.token2id)

{'accept': 0, 'accord': 1, 'achieve': 2, 'act': 3, 'action': 4, 'add': 5, 'additional': 6, 'adjournment': 7, 'adjustment': 8, 'administration': 9, 'administrator': 10, 'afternoon': 11, 'age': 12, 'agree': 13, 'agriculture': 14, 'aid': 15, 'airport': 16, 'aj': 17, 'allen': 18, 'allot': 19, 'allow': 20, 'allowance': 21, 'alpharetta': 22, 'also': 23, 'alternative': 24, 'ambiguous': 25, 'amendment': 26, 'amicable': 27, 'among': 28, 'announce': 29, 'anonymous': 30, 'apparently': 31, 'appoint': 32, 'appointment': 33, 'appraiser': 34, 'approve': 35, 'area': 36, 'arm': 37, 'aside': 38, 'ask': 39, 'asking': 40, 'assistance': 41, 'assistant': 42, 'association': 43, 'atlanta': 44, 'attend': 45, 'attorney': 46, 'audience': 47, 'authorities': 48, 'authority': 49, 'automobile': 50, 'available': 51, 'awarding': 52, 'back': 53, 'ballot': 54, 'bar': 55, 'barber': 56, 'battle': 57, 'become': 58, 'begin': 59, 'belief': 60, 'bellwood': 61, 'berry': 62, 'best': 63, 'birth': 64, 'bit': 65, 'blue': 66, 'boar

In [13]:
print(corpus_[0][:10])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 4), (5, 3), (6, 2), (7, 2), (8, 1), (9, 2)]


In [14]:
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', "atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
[(119, 1), (162, 1), (180, 1), (213, 1), (216, 1), (236, 1), (274, 1), (286, 1), (383, 1), (404, 1), (435, 1)]


In [20]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=15,
                                           id2word=dictionary,
                                           alpha=0.1,
                                           eta=0.1,
                                          )

In [21]:
topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.006*"one" + 0.006*"?" + 0.006*"say" + 0.005*"would" + 0.004*"make" + 0.003*"new" + 0.003*"state" + 0.003*"may" + 0.003*"could" + 0.003*"two" + 0.003*"first" + 0.003*"!" + 0.003*"go" + 0.003*"time" + 0.002*"take"')
(1, '0.006*"would" + 0.005*"one" + 0.004*"?" + 0.004*"make" + 0.003*"say" + 0.003*"new" + 0.003*"could" + 0.003*"go" + 0.003*"two" + 0.003*"use" + 0.003*"time" + 0.003*"like" + 0.003*"may" + 0.003*"state" + 0.002*"even"')
(2, '0.014*"?" + 0.008*"one" + 0.005*"would" + 0.005*"say" + 0.004*"make" + 0.004*"!" + 0.003*"new" + 0.003*"time" + 0.003*"two" + 0.003*"could" + 0.003*"know" + 0.003*"go" + 0.003*"use" + 0.003*"back" + 0.003*"first"')
(3, '0.009*"?" + 0.007*"one" + 0.005*"would" + 0.004*"say" + 0.004*"!" + 0.004*"time" + 0.003*"state" + 0.003*"make" + 0.003*"may" + 0.003*"first" + 0.003*"go" + 0.003*"get" + 0.002*"see" + 0.002*"use" + 0.002*"two"')
(4, '0.007*"one" + 0.007*"?" + 0.007*"say" + 0.006*"would" + 0.004*"new" + 0.004*"make" + 0.003*"could" + 0.003*"take" 

In [22]:
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(1, 0.22774576), (4, 0.096481584), (10, 0.60564), (12, 0.06405684)]
document ID 1:[(1, 0.014057431), (4, 0.9505969), (12, 0.03238276)]
document ID 2:[(1, 0.9905809)]
document ID 3:[(0, 0.36523935), (1, 0.04439435), (2, 0.3123723), (4, 0.10921608), (10, 0.12857193), (12, 0.013846167), (13, 0.016193867)]
document ID 4:[(0, 0.5380465), (1, 0.07532303), (4, 0.04754275), (5, 0.03867259), (7, 0.08378406), (12, 0.21578543)]
document ID 5:[(4, 0.41032654), (10, 0.01504933), (12, 0.57349443)]
document ID 6:[(4, 0.10857097), (5, 0.8235025), (12, 0.0463364), (13, 0.014535768)]
document ID 7:[(0, 0.4702088), (1, 0.16711047), (4, 0.31870663), (5, 0.04291149)]
document ID 8:[(1, 0.5259009), (4, 0.12441394), (12, 0.34857216)]
document ID 9:[(0, 0.010442254), (4, 0.33968523), (5, 0.1403154), (10, 0.50851667)]


In [18]:
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [26]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(1, 0.21654949), (4, 0.10442918), (10, 0.5988123), (12, 0.073717006)]
['news']
The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate a

In [34]:
count = []
for i in range(15):
  count.append(0)

for i in range(500):
  max_c = 0
  max_l = 0
  for t in ldamodel.get_document_topics(corpus_[i]):
    if t[1]>max_l:
      max_c = t[0]
      max_l = t[1]
  count[max_c] += 1

print(count)

[41, 31, 58, 12, 77, 88, 5, 7, 10, 22, 11, 63, 33, 31, 11]


In [39]:
#先頭44文書
count = []
for i in range(15):
  count.append(0)

for i in range(44):
  max_c = 0
  max_l = 0
  for t in ldamodel.get_document_topics(corpus_[i]):
    if t[1]>max_l:
      max_c = t[0]
      max_l = t[1]
  count[max_c] += 1

print(count)

[4, 8, 1, 2, 9, 12, 1, 0, 0, 0, 3, 1, 1, 0, 2]


In [68]:
def lda(num):

  ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_, num_topics=num, id2word=dictionary, alpha=0.1, eta=0.1)
  count = []
  for i in range(num):
    count.append(0)

  for i in range(500):
    max_c = 0
    max_l = 0
    for t in ldamodel.get_document_topics(corpus_[i]):
      if t[1]>max_l:
        max_c = t[0]
        max_l = t[1]
    count[max_c] += 1
  print(count)

  c = []
  for i in range(num):
    c.append(0)

  for i in range(44):
    max_c = 0
    max_l = 0
    for t in ldamodel.get_document_topics(corpus_[i]):
      if t[1]>max_l:
        max_c = t[0]
        max_l = t[1]
    c[max_c] += 1

  print(c)  

In [69]:
lda(5)

[57, 89, 107, 142, 105]
[6, 7, 6, 21, 4]


In [70]:
lda(10)

[64, 22, 8, 51, 36, 31, 46, 104, 96, 42]
[9, 0, 0, 7, 7, 3, 2, 0, 7, 9]


In [72]:
lda(20)

[19, 37, 19, 19, 29, 21, 76, 41, 13, 16, 34, 3, 25, 50, 49, 5, 18, 9, 7, 10]
[1, 10, 1, 0, 1, 1, 2, 6, 2, 2, 4, 0, 1, 3, 8, 0, 0, 1, 1, 0]
